Bot con corpus de wikipedia

In [1]:
import json
import string
import random
import re
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [15]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Cafe')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [16]:
article_text

'el café es la bebida que se obtiene a partir de los granos tostados y molidos de los frutos de la planta del café (cafeto); es altamente estimulante por su contenido de cafeína,[1]\u200b una sustancia psicoactiva.[2]\u200b es uno de los productos más comercializados a nivel mundial, además de estar entre las tres bebidas más consumidas por el ser humano (junto con el agua y el té).[1]\u200b suele tomarse durante o después del desayuno, y puede llegar a ser considerado para muchas personas como único desayuno, aunque también se toma en diferentes ocasiones y a diversas horas del día, como en la merienda, después del almuerzo o cena; y es ideal para entablar conversaciones, y en muchos países por costumbre o tradición se reparte en reuniones familiares como funerales; es por ello que es valorada en diferentes países del mundo como una de las bebidas sin alcohol más socializadoras que existen. el gusto por el café no siempre ha sido espontáneo en las culturas mundiales y  las nuevas gene

In [17]:
print("Cantidad de caracteres en la nota:", len(article_text))


Cantidad de caracteres en la nota: 69334


Preprocessing

In [18]:
text = re.sub(r'\[[0-9]*\]', ' ', article_text)
text = re.sub(r'\s+', ' ', text)

In [19]:
text

'el café es la bebida que se obtiene a partir de los granos tostados y molidos de los frutos de la planta del café (cafeto); es altamente estimulante por su contenido de cafeína, \u200b una sustancia psicoactiva. \u200b es uno de los productos más comercializados a nivel mundial, además de estar entre las tres bebidas más consumidas por el ser humano (junto con el agua y el té). \u200b suele tomarse durante o después del desayuno, y puede llegar a ser considerado para muchas personas como único desayuno, aunque también se toma en diferentes ocasiones y a diversas horas del día, como en la merienda, después del almuerzo o cena; y es ideal para entablar conversaciones, y en muchos países por costumbre o tradición se reparte en reuniones familiares como funerales; es por ello que es valorada en diferentes países del mundo como una de las bebidas sin alcohol más socializadoras que existen. el gusto por el café no siempre ha sido espontáneo en las culturas mundiales y las nuevas generacione

Division en sentencias y palabras

In [20]:
corpus = nltk.sent_tokenize(text)
words = nltk.word_tokenize(text)

In [21]:
corpus[:10]


['el café es la bebida que se obtiene a partir de los granos tostados y molidos de los frutos de la planta del café (cafeto); es altamente estimulante por su contenido de cafeína, \u200b una sustancia psicoactiva.',
 '\u200b es uno de los productos más comercializados a nivel mundial, además de estar entre las tres bebidas más consumidas por el ser humano (junto con el agua y el té).',
 '\u200b suele tomarse durante o después del desayuno, y puede llegar a ser considerado para muchas personas como único desayuno, aunque también se toma en diferentes ocasiones y a diversas horas del día, como en la merienda, después del almuerzo o cena; y es ideal para entablar conversaciones, y en muchos países por costumbre o tradición se reparte en reuniones familiares como funerales; es por ello que es valorada en diferentes países del mundo como una de las bebidas sin alcohol más socializadoras que existen.',
 'el gusto por el café no siempre ha sido espontáneo en las culturas mundiales y las nueva

In [22]:
words[:10]

['el', 'café', 'es', 'la', 'bebida', 'que', 'se', 'obtiene', 'a', 'partir']

In [23]:
print("Vocabulario:", len(words))

Vocabulario: 12946


In [24]:
#Lemmatizacion

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula
    # 2 - quitar los simbolos de puntuacion
    # 3 - realiza la tokenización
    # 4 - realiza la lematización
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [26]:
#tfidf

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "Lo siento, no te entendí"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

In [27]:
import sys
!{sys.executable} -m pip install gradio --quiet

     |████████████████████████████████| 11.6 MB 7.2 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 2.3 MB 42.7 MB/s 
     |████████████████████████████████| 213 kB 47.2 MB/s 
     |████████████████████████████████| 106 kB 53.7 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 278 kB 54.6 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 64 kB 2.2 MB/s 
     |████████████████████████████████| 80 kB 8.1 MB/s 
     |████████████████████████████████| 68 kB 6.0 MB/s 
     |████████████████████████████████| 68 kB 5.8 MB/s 
     |████████████████████████████████| 68 kB 6.2 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 4.1 MB 40.5 MB/s 
     |██████████████████████████████

In [ ]:
import gradio as gr

def bot_response(human_text):
    print("Q:", human_text)    
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `layout` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Q:  que es el cafeto?


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


A: el cafeto es un arbusto de 10 a 12 metros que crece espontáneamente.
Q: descafeinado
A: el cloruro de metileno se reutiliza para posteriores procesos de descafeinado.
Q: Cual es la acidez
A: es de sabor intenso, con mayor contenido de cafeína (dos veces más que el arábigo ​) y con mayor acidez.
Q: Origen del café
A: los granos del café son uno de los principales productos de origen agrícola que se comercializan en los mercados internacionales y a menudo suponen una gran contribución a los rubros de exportación de las regiones productoras.
Q: Origen del café
A: los granos del café son uno de los principales productos de origen agrícola que se comercializan en los mercados internacionales y a menudo suponen una gran contribución a los rubros de exportación de las regiones productoras.
Q: especies de café
A: existen dos grandes especies genéticas o especies, cafeto arabica (coffea arabica) y cafeto robusta (coffea canephora).
Q: café en europa
A: en el sur y oeste de europa se observó 